In [348]:
import pandas as pd
import numpy as np 
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import nltk
nltk.download('punkt')#
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet
nltk.download('wordnet')#
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords') #
import sys  
!{sys.executable} -m pip install pyspellchecker 
from spellchecker import SpellChecker 

[nltk_data] Downloading package punkt to /Users/Tobias/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Tobias/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/Tobias/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Tobias/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [349]:
train = pd.read_csv('train.csv',encoding ="utf-8" )
#pd.set_option('display.max_colwidth', -1)

In [350]:
#train.drop(["id"], axis=1, inplace=True)

In [351]:
train["text"]=train["text"].str.lower()

In [352]:
trainS = train
trainS.head(20)

,keyword,location,text,target
0,NaN,NaN,our deeds are the reason of this #earthquake may allah forgive us all,1
1,NaN,NaN,forest fire near la ronge sask. canada,1
2,NaN,NaN,all residents asked to 'shelter in place' are being notified by officers. no other evacuation or shelter in place orders are expected,1
3,NaN,NaN,"13,000 people receive #wildfires evacuation orders in california",1
4,NaN,NaN,just got sent this photo from ruby #alaska as smoke from #wildfires pours into a school,1
5,NaN,NaN,#rockyfire update => california hwy. 20 closed in both directions due to lake county fire - #cafire #wildfires,1
6,NaN,NaN,"#flood #disaster heavy rain causes flash flooding of streets in manitou, colorado springs areas",1
7,NaN,NaN,i'm on top of the hill and i can see a fire in the woods...,1
8,NaN,NaN,there's an emergency evacuation happening now in the building across the street,1
9,NaN,NaN,i'm afraid that the tornado is coming to our area...,1


In [353]:
Contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"gonna": "going to",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [354]:
def data_text_preprocess(total_text):
        
        total_text = re.sub('http\S+',' ', str(total_text))
        total_test = re.sub('https\S+',' ', str(total_text))
        total_text = re.sub('\@\S+',' ', str(total_text))
        total_text = re.sub('\#',' ', str(total_text))
        total_text = re.sub('[^a-zA-Z\n]', ' ', str(total_text))
        total_text = re.sub(r" \B'\b|\b'\B", " ", str(total_text))
        
       
        return total_text

In [355]:
trainS['text'] = trainS['text'].map(data_text_preprocess)
trainS.head(20)

,keyword,location,text,target
0,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1
3,NaN,NaN,people receive wildfires evacuation orders in california,1
4,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1
5,NaN,NaN,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1
6,NaN,NaN,flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas,1
7,NaN,NaN,i m on top of the hill and i can see a fire in the woods,1
8,NaN,NaN,there s an emergency evacuation happening now in the building across the street,1
9,NaN,NaN,i m afraid that the tornado is coming to our area,1


In [356]:
 def decontraction(total_text):
        
        for i in Contractions:
            total_text = total_text.replace( i, Contractions.get(i))
        
        return total_text

In [357]:
trainS['text'] = trainS['text'].map(decontraction)
trainS.head(20)

,keyword,location,text,target
0,NaN,NaN,our deeds are the reason of this earthquake may allah forgive us all,1
1,NaN,NaN,forest fire near la ronge sask canada,1
2,NaN,NaN,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1
3,NaN,NaN,people receive wildfires evacuation orders in california,1
4,NaN,NaN,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1
5,NaN,NaN,rockyfire update california hwy closed in both directions due to lake county fire cafire wildfires,1
6,NaN,NaN,flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas,1
7,NaN,NaN,i m on top of the hill and i can see a fire in the woods,1
8,NaN,NaN,there s an emergency evacuation happening now in the building across the street,1
9,NaN,NaN,i m afraid that the tornado is coming to our area,1


In [358]:
def get_wordnet_pos(word):
   
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [359]:
lemmatizer = WordNetLemmatizer()

def lemmatization1(total_text):
    total_text = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(total_text)]

    return total_text

In [360]:
trainS['text'] = trainS['text'].map(lemmatization1)
trainS.head(20)

,keyword,location,text,target
0,NaN,NaN,"[our, deed, be, the, reason, of, this, earthquake, may, allah, forgive, u, all]",1
1,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]",1
2,NaN,NaN,"[all, resident, ask, to, shelter, in, place, be, be, notify, by, officer, no, other, evacuation, or, shelter, in, place, order, be, expect]",1
3,NaN,NaN,"[people, receive, wildfire, evacuation, order, in, california]",1
4,NaN,NaN,"[just, get, sent, this, photo, from, ruby, alaska, a, smoke, from, wildfire, pours, into, a, school]",1
5,NaN,NaN,"[rockyfire, update, california, hwy, close, in, both, direction, due, to, lake, county, fire, cafire, wildfire]",1
6,NaN,NaN,"[flood, disaster, heavy, rain, cause, flash, flood, of, street, in, manitou, colorado, spring, area]",1
7,NaN,NaN,"[i, m, on, top, of, the, hill, and, i, can, see, a, fire, in, the, wood]",1
8,NaN,NaN,"[there, s, an, emergency, evacuation, happen, now, in, the, building, across, the, street]",1
9,NaN,NaN,"[i, m, afraid, that, the, tornado, be, come, to, our, area]",1


In [361]:
stop_words = set(stopwords.words('english'))


In [362]:
def remove_stopwords(total_text):
    
    filtered_word_list = total_text[:]
    for word in total_text:
        if word in stop_words: 
            filtered_word_list.remove(word)
    
    #filtered_sentence = [w for w in total_text if not w in stop_words] 
    #filtered_sentence= []
    
    #for w in total_text: 
      #  if w not in stop_words: 
       #     filtered_sentence.append(w) 

    return filtered_word_list

In [363]:
trainS['text'] = trainS['text'].map(remove_stopwords)
trainSR = trainS
trainSR.head(20)

,keyword,location,text,target
0,NaN,NaN,"[deed, reason, earthquake, may, allah, forgive, u]",1
1,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]",1
2,NaN,NaN,"[resident, ask, shelter, place, notify, officer, evacuation, shelter, place, order, expect]",1
3,NaN,NaN,"[people, receive, wildfire, evacuation, order, california]",1
4,NaN,NaN,"[get, sent, photo, ruby, alaska, smoke, wildfire, pours, school]",1
5,NaN,NaN,"[rockyfire, update, california, hwy, close, direction, due, lake, county, fire, cafire, wildfire]",1
6,NaN,NaN,"[flood, disaster, heavy, rain, cause, flash, flood, street, manitou, colorado, spring, area]",1
7,NaN,NaN,"[top, hill, see, fire, wood]",1
8,NaN,NaN,"[emergency, evacuation, happen, building, across, street]",1
9,NaN,NaN,"[afraid, tornado, come, area]",1


In [364]:
def removesinglelettertoken(total_text):
    emptylist=[]
    for w in total_text:
        if len(w)>1:
            emptylist.append(w)
    return emptylist

In [365]:
trainSR['text'] = trainSR['text'].map(removesinglelettertoken)

trainSR.head(20)

,keyword,location,text,target
0,NaN,NaN,"[deed, reason, earthquake, may, allah, forgive]",1
1,NaN,NaN,"[forest, fire, near, la, ronge, sask, canada]",1
2,NaN,NaN,"[resident, ask, shelter, place, notify, officer, evacuation, shelter, place, order, expect]",1
3,NaN,NaN,"[people, receive, wildfire, evacuation, order, california]",1
4,NaN,NaN,"[get, sent, photo, ruby, alaska, smoke, wildfire, pours, school]",1
5,NaN,NaN,"[rockyfire, update, california, hwy, close, direction, due, lake, county, fire, cafire, wildfire]",1
6,NaN,NaN,"[flood, disaster, heavy, rain, cause, flash, flood, street, manitou, colorado, spring, area]",1
7,NaN,NaN,"[top, hill, see, fire, wood]",1
8,NaN,NaN,"[emergency, evacuation, happen, building, across, street]",1
9,NaN,NaN,"[afraid, tornado, come, area]",1


In [366]:
spell = SpellChecker()

def spellchecker(total_text):
    for w in total_text: 
        spell.correction(w) 
    return total_text

In [367]:
#trainSR['text'] = trainSR['text'].map(spellchecker)

#trainSR.head(20)

In [368]:
# Save to csv
trainSR.to_csv("clean.csv", index=False, encoding = 'utf-8')